In [0]:
import numpy as np
from collections import OrderedDict
import tensorflow as tf

In [0]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling3D, Flatten, Conv3D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.losses import CategoricalCrossentropy
#from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
import random
from os import walk


Actions = np.array([np.array([0, 1, 0]),
np.array([0, 0, 0]),
np.array([0, 0, 0.4]),
np.array([-1, 0, 0]),
np.array([1, 0, 0]),
np.array([-1, 0, 0.4]),
np.array([1, 0, 0.4])])

In [0]:
walking = walk('/content/drive/My Drive/Gym')
for i in walking:
  names = i[2]
  break

In [4]:
import pandas as pd

states_expanded = []
actions_expanded = []
actions_one_hot_expanded = []
v = 0
for j in names:
  data = pd.read_pickle('/content/drive/My Drive/Gym/{}'.format(j))
  states = data['states']
  actions = data['actions']
  actions_one_hot = actions_one_hot = data['actions_onehot']
  for i in range(55, len(states)):
    states_expanded.append(np.array([states[i - m] for m in range(5)]))
    actions_expanded.append(np.array(actions[i]))
    actions_one_hot_expanded.append(np.array(actions_one_hot[i]))
actions_expanded = np.array(actions_expanded)
actions_one_hot_expanded = np.array(actions_one_hot_expanded)
print(len(states_expanded))

95541


In [0]:
X_train = np.array(states_expanded[:50000])
del states_expanded[:50000]
X_test = np.array(states_expanded[:30000])
del states_expanded[:30000]
X_train = np.append(X_train, X_test, axis=0)
X_test = None

In [0]:
X_test = np.array(states_expanded)
states_expanded = None

In [0]:
from sys import getsizeof

In [0]:
one_hot = np.array([i for i in range(7)]).reshape(-1,1)

In [0]:
data = None
states = None
actions = None
actions_one_hot = None
# states_expanded = np.array(states_expanded)

In [10]:
from sklearn.preprocessing import OneHotEncoder

OH = OneHotEncoder()
OH.fit(one_hot)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [0]:
def create_model():
  inputs = Input(shape=(5,96,96,3))
  X = Conv3D(filters=32, kernel_size=(1,5,5), padding='same', activation='relu')(inputs)
  X = MaxPooling3D((1,2,2))(X)
  X = Conv3D(filters=64, kernel_size=(1,5,5), padding='same', activation='relu')(X)
  X = MaxPooling3D((1,2,2))(X)
  # X = Conv3D(filters=128, kernel_size=(1,5,5), padding='same', activation='relu')(X)
  # X = MaxPooling3D((1,2,2))(X)
  X = Flatten()(X)
  X = Dense(64, activation='relu')(X)
  outputs = Dense(7, activation='softmax')(X)
  model = Model(inputs=inputs,outputs=outputs)
  return model

In [12]:
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5, 96, 96, 3)]    0         
_________________________________________________________________
conv3d (Conv3D)              (None, 5, 96, 96, 32)     2432      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 5, 48, 48, 32)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 5, 48, 48, 64)     51264     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 5, 24, 24, 64)     0         
_________________________________________________________________
flatten (Flatten)            (None, 184320)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                117965

In [0]:
model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(), metrics=['categorical_crossentropy'])

In [0]:
history = model.fit(X_train, actions_one_hot_expanded[:80000,:], validation_data=(X_test,actions_one_hot_expanded[80000:,:]),  batch_size=64, epochs=2, shuffle=True)

Epoch 1/2
1250/1250 [==============================] - 578s 463ms/step - loss: 7.8190 - categorical_crossentropy: 7.8190 - val_loss: 0.8021 - val_categorical_crossentropy: 0.8021
Epoch 2/2
 858/1250 [===================>..........] - ETA: 2:49 - loss: 0.7407 - categorical_crossentropy: 0.7407

In [0]:
model.save('/content/drive/My Drive/Gym2/my_model.h5')